In [198]:
import numpy as np
import scanpy as sc

In [199]:
import scanpy as sc

In [169]:
adata = sc.read_h5ad('/space/grp/aadrian/Pseudobulk_Function_Pipeline_HighRes/bin/preprocessing/preprocessSC_pipe/data/h5ad_datasets/processed/withStats/brain_sc_with_metadata_cpm_pc_stats.h5ad')

In [241]:
adata.X

array([[-0.08634007, -0.03052081, -0.05556148, ..., -0.01230601,
        -0.29016137, -0.01911202],
       [-0.11159052, -0.04629401, -0.08018494, ..., -0.01395351,
        -0.31046838, -0.01677084],
       [-0.1100802 , -0.05275587, -0.08634064, ..., -0.0147281 ,
        -0.31433898, -0.01513948],
       [-0.11324341, -0.03069228, -0.06466114, ..., -0.01210005,
        -0.30037493, -0.02059661],
       [-0.07416604, -0.0265131 , -0.04739521, ..., -0.01193582,
        -0.28284082, -0.01938034]], dtype=float32)

In [240]:
sc.pp.subsample(adata, n_obs = 5)
adata

AnnData object with n_obs × n_vars = 5 × 1973
    obs: 'cluster', 'umap_x', 'umap_y', 'Tissue', 'Cluster', 'Cell type', 'Cell type group', 'Cell count', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'norm_factor', 'leiden'
    var: 'hgnc_symbol', 'uniprotswissprot', 'entrezgene_id', 'description', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [236]:
adata2 =adata[adata.obs.loc[:,"Cell type"] == "Inhibitory neurons"]

In [237]:
adata2

View of AnnData object with n_obs × n_vars = 23897 × 1973
    obs: 'cluster', 'umap_x', 'umap_y', 'Tissue', 'Cluster', 'Cell type', 'Cell type group', 'Cell count', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'norm_factor', 'leiden'
    var: 'hgnc_symbol', 'uniprotswissprot', 'entrezgene_id', 'description', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [99]:
import random
import pandas as pd

def getNumToSubsample(loCTs:list, totalSampleSize:int) -> pd.Series:
    """
    Get the number of samples to supsample from each cell type
    
    """
    group_samples = []  # List to store the number of samples per group
    remaining_samples = totalSampleSize  # Initialize remaining samples to the total number

    # Loop through each group except the last one
    for i, group in enumerate(loCTs[:-1]):
        # Generate a random percentage between 0 and the remaining samples divided by the number of remaining groups
        percent = random.uniform(0, remaining_samples / (len(loCTs) - i))
        num_samples = int(percent)  # Convert the percentage to an integer number of samples
        group_samples.append(num_samples)  # Add the number of samples to the list
        remaining_samples -= num_samples  # Subtract the allocated samples from the remaining

    # Allocate the remaining samples to the last group
    group_samples.append(remaining_samples)
    
    
    group_series = pd.Series(group_samples, index = loCTs, name = "samples")
    
    return group_series

# Define the groups and their initial percentage compositions
loCTs = ['Group A', 'Group B', 'Group C', 'Group D']

print(getNumToSubsample(loCTs, 100))

Group A    14
Group B    20
Group C    17
Group D    49
Name: samples, dtype: int64


In [100]:
initial_ratio = 100 / 4 

In [231]:
import random

def getNumberToSubsample(loCTs:list, totalSampleSize:int, variance_factor:float) -> pd.Series:
    """
    Define the umber of cell for each cell type to subsample
    """
    loNumberOfSamples = []  # List to store the number of samples per group
    remaining_samples = totalSampleSize  # Initialize remaining samples to the total number

    # Loop through each group except the last one
    for i, group in enumerate(loCTs[:-1]):
        # Generate a random percentage within a certain range influenced by variance_factor
        min_samples = max(0, remaining_samples / (len(loCTs) - i) - variance_factor * totalSampleSize)
        max_samples = min(remaining_samples / (len(loCTs) - i) + variance_factor * totalSampleSize, remaining_samples)
        
        # Get the number of cells we will sample
        samples = int(random.uniform(min_samples, max_samples))
        
        # print(f'min {min_samples}')
        # print(f'max {max_samples}')
        # print(f'per {samples}')
        
        # turn it into an int
        num_samples = int(samples)
        
        # Add the number of samples we will sample to the list
        loNumberOfSamples.append(num_samples) 
    
        # Remove these samples
        remaining_samples -= num_samples  # Subtract the allocated samples from the remaining

    # Allocate the remaining samples to the last group so that the last group is there
    loNumberOfSamples.append(remaining_samples)
    
    random.shuffle(loNumberOfSamples)  # Shuffle the order of allocated samples among groups
    
    return pd.Series(loNumberOfSamples, index = loCTs, name = 'numbers_to_sample')

# Define the groups and their initial percentage compositions
loCTs = ['Group A', 'Group B', 'Group C', 'Group D']

# Total number of samples
totalSampleSize = 100
variance_factor = 0.05
getNumberToSubsample(loCTs, totalSampleSize, variance_factor)

Group A    27
Group B    20
Group C    24
Group D    29
Name: numbers_to_sample, dtype: int64

In [171]:
data = pd.Series([10, 20, 30, 40, 50], index=['a', 'b', 'c', 'd', 'e'], name="MySeries")

In [197]:
for i, n_cells in enumerate(data):
    print(data.index[i])
    print(n_cells)

a
10
b
20
c
30
d
40
e
50
